Ricerca e controllo per possibili outlier

Import delle librerie necessarie


In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# ---------------------------------------------------------
# Nota per i colleghi (Serena): normalizzo qui le colonne Price e Annual Income
# - leggo il file cleaned (usa CLEANED_PATH se già definito nel notebook)
# - pulisco "Price ($)" e "Annual Income" in colonne numeriche _price_clean e _income_clean
# - creo le colonne canonicali 'Price' e 'Annual Income' così il resto del notebook non va toccato
# ---------------------------------------------------------
import os
import pandas as pd

# percorso file (usa CLEANED_PATH se è stato definito altrove nel notebook)
try:
    CLEANED_PATH
except NameError:
    CLEANED_PATH = os.path.join("data", "processed", "database_cleaned.csv")

print(f"Carico il cleaned file da: {CLEANED_PATH}")
if not os.path.exists(CLEANED_PATH):
    raise FileNotFoundError(f"Non trovo {CLEANED_PATH}. Assicuratevi che il file esista o che il path sia corretto.")

# carico e mostro shape/prime colonne per controllo rapido
df_tmp = pd.read_csv(CLEANED_PATH, low_memory=False)
print("Shape:", df_tmp.shape)
print("Colonne disponibili:", list(df_tmp.columns))

# colonna prezzo nota dal dataset
price_col = "Price ($)"
# possibili nomi per il reddito annuale (se presenti)
income_candidates = [c for c in df_tmp.columns if "annual income" in c.lower() or "income" in c.lower()]

# controllo presenza colonne fondamentali
if price_col not in df_tmp.columns:
    raise KeyError(f"Attenzione: mi aspettavo la colonna {price_col!r} nel cleaned dataset ma non è presente. Colonne trovate: {list(df_tmp.columns)}")

# funzione di pulizia standard (rimuove $, virgole, spazi e converte a numerico)
def to_numeric_clean(s):
    return pd.to_numeric(s.astype(str).str.replace(r"[\$,]", "", regex=True).str.replace(r"\s+", "", regex=True), errors="coerce")

# applico la pulizia al prezzo
df_tmp["_price_clean"] = to_numeric_clean(df_tmp[price_col])

# se troviamo una colonna income la puliamo, altrimenti creiamo la colonna con NaN
if income_candidates:
    # scegliamo la prima candidata rilevata
    income_col = income_candidates[0]
    df_tmp["_income_clean"] = to_numeric_clean(df_tmp[income_col])
    print(f"Usata colonna reddito individuata: {income_col}")
else:
    income_col = None
    df_tmp["_income_clean"] = pd.NA
    print("Nessuna colonna reddito trovata automaticamente; _income_clean sarà vuota (NaN).")

# report rapido per verificare che la conversione sia andata a buon fine
print("\n_price_clean non-null:", int(df_tmp["_price_clean"].notna().sum()),
      "  nulls:", int(df_tmp["_price_clean"].isna().sum()))
print("_income_clean non-null:", int(df_tmp["_income_clean"].notna().sum()),
      "  nulls:", int(df_tmp["_income_clean"].isna().sum()))

display(df_tmp[[price_col, "_price_clean"]].head(6))
if income_col:
    display(df_tmp[[income_col, "_income_clean"]].head(6))

print("\n_statistiche _price_clean:")
print(df_tmp["_price_clean"].describe())

# creo colonne canonicali che il resto del notebook si aspetta
df_tmp["Price"] = df_tmp["_price_clean"]
df_tmp["Annual Income"] = df_tmp["_income_clean"]

# rendo il dataframe disponibile con nome atteso dalle celle successive
df_clean = df_tmp

print("\nFatto: df_clean pronto. Uso df_clean['Price'] e df_clean['Annual Income'] per le celle successive.")

Carico il cleaned file da: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/database_cleaned.csv
Shape: (23906, 13)
Colonne disponibili: ['Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Dealer_Region']
Usata colonna reddito individuata: Annual Income

_price_clean non-null: 23906   nulls: 0
_income_clean non-null: 23906   nulls: 0


,Price ($),_price_clean
0,26000,26000
1,19000,19000
2,31500,31500
3,14000,14000
4,24500,24500
5,12000,12000


,Annual Income,_income_clean
0,13500,13500
1,1480000,1480000
2,1035000,1035000
3,13500,13500
4,1465000,1465000
5,850000,850000



_statistiche _price_clean:
count    23906.000000
mean     28090.247846
std      14788.687608
min       1200.000000
25%      18001.000000
50%      23000.000000
75%      34000.000000
max      85800.000000
Name: _price_clean, dtype: float64

Fatto: df_clean pronto. Uso df_clean['Price'] e df_clean['Annual Income'] per le celle successive.


Inizio della parte di ricerca

In [20]:
# ---------------------------------------------------------
# Serena -> caricamento pulito + normalizzazione Price/Income
# - cerco il cleaned file in percorsi standard
# - leggo il file in modo robusto
# - pulisco "Price ($)" in _price_clean e creo la colonna canonical 'Price'
# - pulisco eventuale colonna reddito e creo 'Annual Income'
# - stampo report sintetico per controllo rapido
# ---------------------------------------------------------
import os
import pandas as pd

# definizione percorsi: cerco il cleaned file in vari posti standard del progetto
ROOT = os.path.abspath(os.getcwd())
DATA_DIR = os.path.join(ROOT, "data")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")

candidates = [
    os.path.join(DATA_DIR, "database_cleaned_2.csv"),
    os.path.join(PROCESSED_DIR, "database_cleaned_2.csv"),
    os.path.join(PROCESSED_DIR, "database_cleaned.csv"),
    os.path.join(DATA_DIR, "database_cleaned.csv"),
]

# prendo CLEANED_PATH predefinito se è già stato definito altrove
try:
    CLEANED_PATH
except NameError:
    CLEANED_PATH = None

if not CLEANED_PATH:
    CLEANED_PATH = next((p for p in candidates if os.path.exists(p)), None)

if not CLEANED_PATH:
    raise FileNotFoundError(
        "Nessun file 'database_cleaned' trovato. Controlla dove Matteo ha salvato il file o rigenera il cleaned dataset. "
        "Cercati questi percorsi: " + ", ".join(candidates)
    )

print(f"[serena] Using cleaned file: {CLEANED_PATH}")
df_clean = pd.read_csv(CLEANED_PATH, low_memory=False)

# mostro shape e colonne per immediatezza
print("[serena] Shape:", df_clean.shape)
print("[serena] Columns:", list(df_clean.columns))

# funzione di pulizia robusta per numerici che possono contenere $ , spazi, virgole
def to_numeric_clean(series):
    return pd.to_numeric(series.astype(str).str.replace(r"[\$,]", "", regex=True).str.replace(r"\s+", "", regex=True), errors="coerce")

# costruiamo _price_clean a partire da "Price ($)" se presente, altrimenti cerchiamo alternative
if "_price_clean" not in df_clean.columns:
    if "Price ($)" in df_clean.columns:
        src = "Price ($)"
    else:
        # fallback: trova prima colonna che contiene 'price' o '$'
        cand = [c for c in df_clean.columns if 'price' in c.lower() or '$' in c]
        src = cand[0] if cand else None

    if src is None:
        print("[serena] ERRORE: nessuna colonna prezzo trovata automaticamente. Colonne disponibili:")
        print(list(df_clean.columns))
        raise KeyError("Nessuna colonna prezzo trovata (cerco 'Price ($)' o colonne con 'price').")
    print(f"[serena] Building _price_clean from column: {src}")
    df_clean["_price_clean"] = to_numeric_clean(df_clean[src])
else:
    print("[serena] _price_clean già presente, lo uso così com'è.")

# assicuriamo la colonna canonical 'Price' per la compatibilità con le celle seguenti
if "Price" not in df_clean.columns or df_clean["Price"].isna().sum() > 0:
    df_clean["Price"] = df_clean["_price_clean"]
    print("[serena] Impostata/aggiornata colonna canonical 'Price' a partire da _price_clean.")

# reddito: cerchiamo colonne candidate e puliamo
income_candidates = [c for c in df_clean.columns if "annual income" in c.lower() or "income" == c.lower() or "income" in c.lower()]
if "_income_clean" not in df_clean.columns:
    if income_candidates:
        inc_src = income_candidates[0]
        print(f"[serena] Building _income_clean from column: {inc_src}")
        df_clean["_income_clean"] = to_numeric_clean(df_clean[inc_src])
        df_clean["Annual Income"] = df_clean["_income_clean"]
    else:
        df_clean["_income_clean"] = pd.NA
        print("[serena] Nessuna colonna reddito trovata automaticamente; _income_clean sarà NaN.")

# report rapido
print("\n[serena] Verifica rapida:")
print(" - non-null _price_clean:", int(df_clean["_price_clean"].notna().sum()), " / totale:", len(df_clean))
print(" - non-null Price:", int(df_clean["Price"].notna().sum()), " / totale:", len(df_clean))
if "_income_clean" in df_clean.columns:
    print(" - non-null _income_clean:", int(df_clean["_income_clean"].notna().sum()))

display(df_clean[[c for c in ["Price ($)", "Price", "_price_clean"] if c in df_clean.columns]].head(8))

print("\n[serena] df_clean pronto: le celle successive possono ora usare df_clean['Price'] e df_clean['Annual Income'] senza KeyError.")

[serena] Using cleaned file: /Users/serenatempesta/Documents/Progetti/Data_Analysis/progetto_finale/data/processed/database_cleaned.csv
[serena] Shape: (23906, 13)
[serena] Columns: ['Date', 'Customer Name', 'Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Dealer_Region']
[serena] Building _price_clean from column: Price ($)
[serena] Impostata/aggiornata colonna canonical 'Price' a partire da _price_clean.
[serena] Building _income_clean from column: Annual Income

[serena] Verifica rapida:
 - non-null _price_clean: 23906  / totale: 23906
 - non-null Price: 23906  / totale: 23906
 - non-null _income_clean: 23906


,Price ($),Price,_price_clean
0,26000,26000,26000
1,19000,19000,19000
2,31500,31500,31500
3,14000,14000,14000
4,24500,24500,24500
5,12000,12000,12000
6,14000,14000,14000
7,42000,42000,42000



[serena] df_clean pronto: le celle successive possono ora usare df_clean['Price'] e df_clean['Annual Income'] senza KeyError.


In [21]:
outlier_alti = df_clean[df_clean['Price'] > 80000]
outlier_bassi = df_clean[df_clean['Price'] < 9000]
num_outlier_alti = len(outlier_alti)
num_outlier_bassi = len(outlier_bassi)
print(f"\nNumero di outlier con prezzo > 80000: {num_outlier_alti}")
print(f"Numero di outlier con prezzo < 9000: {num_outlier_bassi}")
df_no_outliers = df_clean[(df_clean['Price'] <= 80000) & (df_clean['Price'] >= 9000)]
print(f"\nDimensione del dataset originale: {len(df_clean)}")
print(f"Dimensione del dataset senza outlier: {len(df_no_outliers)}")


Numero di outlier con prezzo > 80000: 220
Numero di outlier con prezzo < 9000: 6

Dimensione del dataset originale: 23906
Dimensione del dataset senza outlier: 23680


In [22]:
df_clean.max()['Annual Income']
df_clean.min()['Annual Income']
df_clean['Annual Income'].mean()
df_clean['Annual Income'].median()
print("Media:", df_clean['Annual Income'].mean())
print("Mediana:", df_clean['Annual Income'].median())
print("Massimo:",df_clean.max()['Annual Income'])
print("Minimo:",df_clean.min()['Annual Income'])
top_10_income = df_clean.nlargest(10, 'Annual Income')
print(top_10_income[['Annual Income']])
bottom_10_income = df_clean.nsmallest(10, 'Annual Income')
print(bottom_10_income[['Annual Income']])



Media: 830840.2851167071
Mediana: 735000.0
Massimo: 11200000
Minimo: 10080
       Annual Income
14026       11200000
15675        8000000
6150         7650000
9996         6800000
22407        6600000
11607        6500000
7657         6460000
8817         6400000
14183        6400000
4755         6240000
       Annual Income
23451          10080
0              13500
3              13500
7              13500
9              13500
10             13500
11             13500
13             13500
20             13500
28             13500


In [23]:
outlier_salari_alti = df_clean[df_clean['Annual Income'] > 7000000]
outlier_salari_bassi = df_clean[df_clean['Annual Income'] < 100000]
num_outlier_salari_alti = len(outlier_salari_alti)
num_outlier_salari_bassi = len(outlier_salari_bassi)
print(f"\nNumero di outlier con salari > 7000000: {num_outlier_salari_alti}")
print(f"Numero di outlier con salari < 100000: {num_outlier_salari_bassi}")
df_no_outliers = df_clean[(df_clean['Annual Income'] <= 7000000) & (df_clean['Annual Income'] >= 100000)]
print(f"\nDimensione del dataset originale: {len(df_clean)}")
print(f"Dimensione del dataset senza outlier: {len(df_no_outliers)}")


Numero di outlier con salari > 7000000: 3
Numero di outlier con salari < 100000: 5276

Dimensione del dataset originale: 23906
Dimensione del dataset senza outlier: 18627


In [24]:
top_10_redditi = df_clean.sort_values(by='Annual Income', ascending=False).head(10)
print("\nLe 10 auto con il reddito annuale più alto:")
print(top_10_redditi[['Company', 'Model', 'Annual Income', 'Price']])
bottom_10_redditi = df_clean.sort_values(by='Annual Income', ascending=False).tail(10)
print("\nLe 10 auto con il reddito annuale più basso:")
print(bottom_10_redditi[['Company', 'Model', 'Annual Income', 'Price']])



Le 10 auto con il reddito annuale più alto:
          Company          Model  Annual Income  Price
14026  Oldsmobile        Bravada       11200000  26001
15675  Mercedes-B        S-Class        8000000  85000
6150      Hyundai         Sonata        7650000  21000
9996          BMW           323i        6800000  15000
22407     Mercury          Sable        6600000  39000
11607        Ford        Mustang        6500000  25000
7657       Toyota         Celica        6460000  14000
8817      Mercury  Grand Marquis        6400000  71000
14183      Nissan          Quest        6400000  32001
4755     Chrysler       Concorde        6240000  42000

Le 10 auto con il reddito annuale più basso:
          Company        Model  Annual Income  Price
13099  Volkswagen       Passat          13500  21000
13097   Chevrolet     Cavalier          13500  20000
13096  Mercedes-B        CL500          13500  22000
13095        Ford     Explorer          13500  22000
13091    Chrysler          LHS         

In [25]:
top_10_auto = df_clean.sort_values(by='Price', ascending=False).head(10)
print("\nLe 10 auto con il prezzo più alto:")
print(top_10_auto[['Company', 'Model', 'Annual Income', 'Price']])
bottom_10_auto = df_clean.sort_values(by='Price', ascending=False).tail(10)
print("\nLe 10 auto con il prezzo più basso:")
print(bottom_10_auto[['Company', 'Model', 'Annual Income', 'Price']])


Le 10 auto con il prezzo più alto:
          Company     Model  Annual Income  Price
7068     Cadillac  Eldorado        1388000  85800
17129    Cadillac  Eldorado        5046000  85601
13605    Cadillac  Eldorado        1036000  85600
358        Toyota      RAV4        1326000  85600
9228         Audi        A6         497500  85500
11330    Cadillac  Eldorado        1185000  85500
17947    Cadillac  Eldorado        1414000  85400
2661     Cadillac  Eldorado        1483000  85301
11428      Toyota      RAV4        1053000  85300
6530   Mercedes-B   S-Class         392250  85250

Le 10 auto con il prezzo più basso:
          Company     Model  Annual Income  Price
12717  Volkswagen    Passat          13500   9000
20711  Volkswagen    Passat         522000   9000
23247  Volkswagen    Passat        1900000   9000
8081     Plymouth      Neon         880000   9000
14185        Ford    Taurus        2200000   4300
13949        Ford  Explorer         680000   4200
14020  Mercedes-B     CL500

In [ ]:
# ---------------------------------------------------------
# Serena -> applicazione mapping categorical + pulizia base
# - leggo eventuali mapping generati in notebook/mappings/*.csv
# - normalizzo (trim, lowercase) e applico canonical dove disponibile
# - mantengo raw->canonical fallback se mapping vuoti
# ---------------------------------------------------------
import os
import pandas as pd

# controllo df_clean in memoria
if 'df_clean' not in globals():
    raise RuntimeError("df_clean non esiste. Esegui prima la cella di caricamento/normalizzazione.")

MAP_DIR = os.path.join("notebook", "mappings")
os.makedirs(MAP_DIR, exist_ok=True)

# colonne categorical da normalizzare (quelle che avevamo analizzato)
cat_cols = ['Company', 'Dealer_Name', 'Model', 'Transmission', 'Gender', 'Customer Name', 'Dealer_Region', 'city_state']

def canonicalize_column(df, col, map_dir=MAP_DIR):
    """Applica mapping se esiste mapping/<col>_mapping.csv altrimenti esegue trim/lower."""
    map_fn = os.path.join(map_dir, f"{col}_mapping.csv")
    if os.path.exists(map_fn):
        # legge mapping e usa canonical se presente, altrimenti fallback al raw normalizzato
        m = pd.read_csv(map_fn).fillna('')
        # assicuriamo colonne raw/canonical
        if {'raw','canonical'}.issubset(m.columns):
            # build dict (raw normalizzato -> canonical se non vuoto altrimenti raw)
            d = {r.strip().lower(): (c.strip() if c.strip()!='' else r.strip()) for r,c in zip(m['raw'].astype(str), m['canonical'].astype(str))}
            # applica
            df[col + "_mapped"] = df[col].astype(str).str.strip().str.lower().map(d).fillna(df[col].astype(str).str.strip().str.lower())
            return True
    # fallback: trim + lower
    df[col + "_mapped"] = df[col].astype(str).str.strip().str.lower()
    return False

# applica ai cat_cols presenti nel df
for c in cat_cols:
    if c in df_clean.columns:
        used_map = canonicalize_column(df_clean, c)
        print(f"[serena] Colonna {c}: mapping applicato? {used_map}")
    else:
        print(f"[serena] Colonna {c}: non presente nel dataframe, salto.")

# breve controllo
print("\nEsempio valori canonical (per Company, Model, Dealer_Region se esistono):")
for c in ['Company','Model','Dealer_Region']:
    mapped = c + "_mapped"
    if mapped in df_clean.columns:
        print(f" - {mapped}: top 5 ->")
        display(df_clean[mapped].value_counts().head(5))

In [ ]:
# ---------------------------------------------------------
# Serena -> gestione missing, placeholder e outlier (semplice)
# - sostituisco placeholder noti con NaN/John Doe già gestito
# - rimuovo spazi, converto Price e Annual Income a numerico (se non già fatto)
# - gestisco outlier su Price e Annual Income con winsorization 1%-99%
# - stampo riepilogo e distribuzioni sintetiche
# ---------------------------------------------------------
import numpy as np

# placeholder comuni che vogliamo considerare come NaN
placeholders = ['', 'nan', 'none', 'unknown', 'n/a', 'na', 'john doe']

# normalizzo stringhe su tutte le colonne stringa per uniformità (non distruttivo)
for col in df_clean.select_dtypes(include='object').columns:
    df_clean[col] = df_clean[col].astype(str).str.strip()

# assicuriamo colonne numeriche pulite (se già esistono _price_clean / _income_clean le usiamo)
if '_price_clean' not in df_clean.columns:
    # prova a pulire da "Price ($)" o altro
    if "Price ($)" in df_clean.columns:
        src = "Price ($)"
    else:
        src = next((c for c in df_clean.columns if 'price' in c.lower()), None)
    if src:
        df_clean['_price_clean'] = pd.to_numeric(df_clean[src].astype(str).str.replace(r'[\$,]', '', regex=True).str.replace(r'\s+', '', regex=True), errors='coerce')
    else:
        df_clean['_price_clean'] = pd.NA

if '_income_clean' not in df_clean.columns:
    inc_src = next((c for c in df_clean.columns if 'annual income' in c.lower() or 'income' in c.lower()), None)
    if inc_src:
        df_clean['_income_clean'] = pd.to_numeric(df_clean[inc_src].astype(str).str.replace(r'[\$,]', '', regex=True).str.replace(r'\s+', '', regex=True), errors='coerce')
    else:
        df_clean['_income_clean'] = pd.NA

# sostituisci placeholder con NaN
for c in df_clean.columns:
    if df_clean[c].dtype == object:
        df_clean.loc[df_clean[c].str.lower().isin(placeholders), c] = pd.NA

# Outlier: winsorization semplice (1%-99%) su _price_clean e _income_clean
def winsorize_series(s, lower_q=0.01, upper_q=0.99):
    if s.dropna().empty:
        return s
    lo = s.quantile(lower_q)
    hi = s.quantile(upper_q)
    return s.clip(lower=lo, upper=hi)

if df_clean['_price_clean'].notna().sum() > 0:
    df_clean['_price_clean_w'] = winsorize_series(df_clean['_price_clean'])
else:
    df_clean['_price_clean_w'] = df_clean['_price_clean']

if df_clean['_income_clean'].notna().sum() > 0:
    df_clean['_income_clean_w'] = winsorize_series(df_clean['_income_clean'])
else:
    df_clean['_income_clean_w'] = df_clean['_income_clean']

# aggiorno colonne canonical per sicurezza
df_clean['Price'] = df_clean['_price_clean_w']
df_clean['Annual Income'] = df_clean['_income_clean_w']

# report sintetico
print("[serena] Price non-null:", int(df_clean['Price'].notna().sum()), " / ", len(df_clean))
print("[serena] Annual Income non-null:", int(df_clean['Annual Income'].notna().sum()), " / ", len(df_clean))
print("\nPrice summary (after winsorization):")
print(df_clean['Price'].describe())

In [ ]:
# ---------------------------------------------------------
# Serena -> salvo versione finale cleaned per condivisione con il team / visualizzazione
# - creo data/processed (se non esiste)
# - salvo database_cleaned_2.csv
# - stampo quick-check per conferma
# ---------------------------------------------------------
import os

OUT_DIR = os.path.join("data", "processed")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_FN = "database_cleaned_2.csv"
OUT_PATH = os.path.join(OUT_DIR, OUT_FN)

# controllo df_clean
if 'df_clean' not in globals():
    raise RuntimeError("df_clean non esiste. Esegui prima le celle di cleaning.")

# salvo (index=False)
df_clean.to_csv(OUT_PATH, index=False)
print(f"[serena] Saved cleaned dataset here: {OUT_PATH}")

# info quick-check
print("[serena] File size (bytes):", os.path.getsize(OUT_PATH))
print("[serena] Head (preview):")
display(df_clean.head(5))

# suggerimento: stampare qui le colonne finali utili per analisi/visual (es. dealer_region_mapped, city_state)
print("\n[serena] Colonne finali principali:", [c for c in df_clean.columns if c in ['dealer_region_mapped','city_state','Company_mapped','Model_mapped','Price','Annual Income']])
